<a href="https://colab.research.google.com/github/jalyngearries/PlanetScope_TimeSeries_Construction_Analysis/blob/main/GRCA_Planet_Order_C223_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PlanetScope & Google Earth Engine Integration**

### **Part 3:** Order Images with Planet Orders API

Now that I've properly registered my Earth Engine cloud project, I'll use the EE API and Planet Orders API to order, clip, and import an image collection using image IDs that I manually selected in the Planet Explorer web interface.

For this example, I am using the image AOI that I created in the Planet Explorer web app that I stored as a .geojson in Google Drive. You can also upload this file directly into the Google Colab environment. This is just one of 20 (plus) scripts I've written to order each of my image collections.
  - *For my project, this is easiest because of the 'Find and Replace function in the easy-to-navigate Colab environment. Which also ensures I have very organized file paths and storage.*

I'm using 223.5 Mile as the example site in this script. I store it as C223 because Python (and most languages) hate numerics at the beginning of a variable name.


### Step 1: Import Planet API & authenticate Planet account for Orders API

In [ ]:
## Import the Earth Engine API
import ee

## Importing other packages for ordering and delivery
import json
import os
import pathlib
import time
import shapely
import requests
from google.colab import drive
import asyncio

## Install & import the Planet API
!pip install planet
import planet
from planet import Session, DataClient, OrdersClient

## Set up & authenticate session

from getpass import getpass
from planet import Auth
api_key = getpass('Enter your API key:')

auth = Auth.from_key(api_key)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.5 MB/s eta 0:00:00
Enter your API key:··········


### Step 2: Authenticate Earth Engine in Python environment

In [ ]:
## Trigger authentication flow
ee.Authenticate()

## Initialize the library
ee.Initialize(project='ee-jg3648')

## (Optional) Mount Google Drive to Colab environment (where my AOIs are stored)
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Step 3: Define cloud delivery location for images

In [ ]:
## Define a cloud desination configuration object (defining the destination to be GEE)
cloud_config = planet.order_request.google_earth_engine(
    project='ee-jg3648', collection='C223_imgcoll_8band')

## Define delivery configuration
delivery_config = planet.order_request.delivery(cloud_config=cloud_config)

### Step 4: Define AOI to clip images

In [ ]:
with open("/content/gdrive/MyDrive/NAU - PhD INF/Sites/223.5 - 223 Mile Camp AOI.geojson") as f:
  C223_json = json.load(f)

C223_coordinates = C223_json['features'][0]['geometry']['coordinates'][0]

print(C223_coordinates)


[[-113.34723754, 35.78363304], [-113.33711391, 35.78363304], [-113.33711391, 35.79218227], [-113.34723754, 35.79218227], [-113.34723754, 35.78363304]]


In [ ]:
C223_AOI = {
    "type":"Polygon",
    "coordinates": [
        [[-113.34723754, 35.78363304], [-113.33711391, 35.78363304], [-113.33711391, 35.79218227], [-113.34723754, 35.79218227], [-113.34723754, 35.78363304]]
                    ]
}

### Step 5: Define images to order

In [ ]:
C223_images_8band = [
    '20200814_173019_50_220b',
    '20200915_173743_14_222f',
    '20200928_173344_25_220b',
    '20201014_173606_58_2271',
    '20201116_173435_11_2257',
    '20201215_173706_76_2441',
    '20210114_182441_98_2307',
    '20210208_174005_91_225a',
    '20210315_173138_14_2460',
    '20210415_173031_70_2439',
    '20210516_174130_07_222f',
    '20210614_172706_85_2459',
    '20210714_173547_31_225a',
    '20210819_172951_35_2448',
    '20210921_172525_43_2420',
    '20210927_172845_97_2445',
    '20211014_172521_05_2453',
    '20211117_172808_09_2464',
    '20211215_173917_42_227e',
    '20220109_173726_64_2264',
    '20220213_175339_68_248e',
    '20220323_173529_28_225a',
    '20220417_175301_02_247c',
    '20220516_175207_54_2480',
    '20220617_175557_58_247e',
    '20220711_173929_40_2262',
    '20220824_174838_38_222f',
    '20220918_175312_59_2495',
    '20220923_175557_41_247f',
    '20221018_175531_71_2474',
    '20221116_173922_45_2251',
    '20221217_175701_71_249a',
    '20230106_175448_41_2492',
    '20230216_175534_97_2481',
    '20230316_175340_73_2488',
    '20230421_175932_15_2484',
    '20230426_180046_99_241c',
    '20230515_174700_14_2251',
    '20230617_175800_08_2480',
    '20230716_175916_23_2488',
    '20230813_173020_13_24c8',
    '20230916_172434_36_2465',
    '20230921_173004_63_24b9',
    '20231015_173008_26_24a7',
    '20231117_173031_27_24c7',
    '20231214_173020_78_24b9',
    '20240116_173254_23_242e',
    '20240214_173241_70_24cc',
    '20240313_173358_07_24b4',
    '20240416_173116_27_24bb',
    '20240425_173113_17_24b5',
    '20240515_173603_29_24c7',
    '20240615_173601_00_24c5',
    '20240716_173903_27_24af',
    '20240819_173605_66_2417'
]


In [ ]:
C223_images_4band = [
    '20180114_174039_1008',
    '20180206_174037_102f',
    '20180316_174214_1032',
    '20180413_174312_1042',
    '20180515_174441_1014',
    '20180617_174637_0f34',
    '20180715_174632_1002',
    '20180819_174810_1038',
    '20180914_173750_104b',
    '20181017_173251_1048',
    '20181114_174935_1003',
    '20181216_174645_0e2f',
    '20190113_175214_103d',
    '20190210_174602_0e20',
    '20190318_175253_1035',
    '20190424_175400_0f15',
    '20190502_175542_0f42',
    '20190513_175545_1040',
    '20190613_175628_1021',
    '20190714_174701_0e2f',
    '20190814_174649_0e20',
    '20190913_174715_0e3a',
    '20191016_175801_0f17',
    '20191112_175504_1027',
    '20191210_175951_1011',
    '20200115_175815_1012',
    '20200215_175933_101f',
    '20200328_175623_1006',
    '20200418_175732_1032',
    '20200516_175726_1010',
    '20200615_175903_0f28',
    '20200715_175818_1005'
]


### Step 6: Build your order request

In [ ]:
from planet import order_request

item_ids = C223_images_8band

products = [
    order_request.product(item_ids, 'analytic_8b_sr_udm2', 'PSScene')
]

tools = [
    order_request.reproject_tool(projection='EPSG:4326', kernel='cubic'),
    order_request.clip_tool(C223_AOI)
]
request = order_request.build_request(
    'C223_8band_images', products=products, tools=tools, delivery=delivery_config,
)

### Step 7: Create order request

In [ ]:
from planet import Session, OrdersClient

# an async Orders client to request order creation
async def main():
  async with Session(auth=auth) as sess:
    cl = OrdersClient(sess)
    order = await cl.create_order(request)

# async magic to remember: "async def" to create a coroutine, then "await" to make it run
await main()

In [ ]:

        // IMAGE COLLECTION -->  SITE HEALTH DATA

// import image collections
  // 4-Band
var C223_imgcoll_old = ee.ImageCollection('projects/ee-jg3648/assets/C223_imgcoll_4band')
  // 8-Band
var C223_imgcoll = ee.ImageCollection('projects/ee-jg3648/assets/C223_collection')

// import ROIs
var C223_TxPoly_bound_fc = ee.FeatureCollection('projects/ee-jg3648/assets/Polygons/New/C223_TxPoly')

// convert ROI to geometry
var C223_TxPoly = C223_TxPoly_bound_fc.first().geometry();

// define quality bands
var qualitybands = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8']
var qualitybands2 = ['Q1', 'Q2', 'Q3', 'Q4']

// define function to remove bands
var selectBands = function(image){
  return image.select(image.bandNames().removeAll(qualitybands));};
var selectBands2 = function(image){
  return image.select(image.bandNames().removeAll(qualitybands2));};

// apply function to image collection
var C223_imgcoll = C223_imgcoll.map(selectBands)
var C223_imgcoll_old = C223_imgcoll_old.map(selectBands)

// define band names for SuperDove sensors
var bands = {'CoastalBlue': 'B1', 'Blue': 'B2', 'GreenI': 'B3', 'Green': 'B4', 'Yellow': 'B5', 'Red': 'B6', 'RedEdge': 'B7', 'NIR': 'B8'};
var bands2 = {'Blue':'B1', 'Green': 'B2', 'Red': 'B3', 'NIR': 'B4'};

// define vegetation index function parameters
/**
 * Function to calculate and return only the custom vegetation index band for each image in a collection.
 * @param {ee.ImageCollection} imageCollection - The input image collection.
 * @param {String} expression - The formula to compute the index, using band tags (e.g., 'NIR', 'Red').
 * @param {Object} bandNames - Object mapping generic band tags to actual band names in the image.
 * @returns {ee.ImageCollection} - The image collection with only the new index band in each image.
 */

 // create adaptable function to compute the index for each image in a collection
function calculateIndexCollection(imageCollection, expression, bandNames) {
  return imageCollection.map(function(image) {
    var expr = expression.replace(/(B1|B2|B3|B4|B5|B6|B6Edge|B8)/g, function(match) {
      return bandNames[match];

    });
var indexImage = image.expression(expr, {
  'CoastalBlue': image.select(bandNames['CoastalBlue']),
  'Blue': image.select(bandNames['Blue']),
  'GreenI': image.select(bandNames['GreenI']),
  'Green': image.select(bandNames['Green']),
  'Yellow': image.select(bandNames['Yellow']),
  'Red': image.select(bandNames['Red']),
  'RedEdge': image.select(bandNames['RedEdge']),
  'NIR': image.select(bandNames['NIR'])

}).rename('index');
return indexImage;  // Returning only the index band
  });
}

function calculateIndexCollection2(imageCollection, expression, bandNames) {
  return imageCollection.map(function(image) {
    var expr = expression.replace(/(B1|B2|B3|B4)/g, function(match) {
      return bandNames[match];

    });
var indexImage = image.expression(expr, {
  'Blue': image.select(bands2['Blue']),
  'Green': image.select(bands2['Green']),
  'Red': image.select(bands2['Red']),
  'NIR': image.select(bands2['NIR'])

}).rename('index');
return indexImage;  // Returning only the index band
  });
}

// define vegetation index band math expression
var MSAVI_exp = '0.5 * (2 * NIR + 1 - sqrt((2 * NIR + 1) ** 2 - 8 * (NIR - Red)))';

// apply veg index calculation function to image collection
var C223_MSAVI_imgcoll_old = calculateIndexCollection(C223_imgcoll_old, MSAVI_exp, bands2);
var C223_MSAVI_imgcoll = calculateIndexCollection(C223_imgcoll, MSAVI_exp, bands);

// merge image collections into multi-band images
var C223_MSAVI_old = C223_MSAVI_imgcoll_old.toBands();
var C223_MSAVI = C223_MSAVI_imgcoll.toBands();

// define function to change band names to dates
function renameBandsByDate(image) {
  var bandNames = image.bandNames();
  var newBandNames = bandNames.map(function(name) {
    var parts = ee.String(name).split('_');
    return parts.get(0);
  });
  return image.rename(newBandNames);
}

// apply function to change band names to dates
var C223_MSAVI_old = renameBandsByDate(C223_MSAVI_old);
var C223_MSAVI = renameBandsByDate(C223_MSAVI)

// combine MSAVI image stacks for full time series
var C223_MSAVI = ee.Image.cat(C223_MSAVI_old, C223_MSAVI)

// print number of image dates in time series
var numBands = C223_MSAVI.bandNames().size();
print('Number of Image Dates in Time Series:', numBands);

// sample values from image collection inside tx polygon geometry
var C223_TxPoly_MSAVI = C223_MSAVI.sample({region: C223_TxPoly, scale:3, geometries:true})

/*
// export table as CSV to Drive
Export.table.toDrive({
  collection: C223_TxPoly_MSAVI,
  description: 'C223_MSAVI_TimeSeries',
  fileFormat: 'CSV',
  folder:'GEE Exports'
})
*/

          // VISUALIZATION

// viz parameters
  // polygons
var outlineStyle = {color: 'red', width: 1, fillColor: '00000000'};
var C223_TxPoly_bound_fc = C223_TxPoly_bound_fc.style(outlineStyle);

  // map layers
var visparam_timeseries = {"opacity":0.75,"bands":["20190424"],"min":-0.5,"max":0.8,"palette":["f70000","f9ff00","0ac800"]};

// adding to map
Map.addLayer(C223_MSAVI, visparam_timeseries, 'MSAVI Time Series')
Map.addLayer(C223_TxPoly_bound_fc, {}, 'Tx Polygon')

// map parameters
Map.setOptions('SATELLITE')
Map.centerObject(C223_MSAVI, 15)
